In [1]:
from datetime import datetime
import requests
import sys
import numpy as np
import pandas as pd

sys.path.append('..')
import moonlander

from bokeh.io import output_notebook, show, push_notebook
output_notebook()

Loading BokehJS ...

In [2]:
import moonlander as mld
from moonlander.data.providers import Binance

api_key = 'JUY2Z2R1Lwpgta6toGtB8ofCxpiSBEiqqHJM8rp4GCWzIy2kXFbERuvxB2dDD1Cj'
secret_key = 'hDv51FaiLDHEA7Al2zhuG9YlXoKmgntjvgZDofFKNis1hY27x4WXHoL1RIsf1e8F'

binance = Binance(api_key, secret_key)

In [3]:
# from moonlander.data.providers import CoinDCX

# coindcx_api_key = 'fcb1c800d0ee02c93ea14bf61517afa0194d5c695eec9792'
# coindcx_api_secret = '3071e32a059b72505f7d261d95aacb1ba66ab8e4cf5e89214bf329b454196c4c'

# coindcx = CoinDCX(coindcx_api_key, coindcx_api_secret)

In [4]:
btc = moonlander.core.Asset('DOGE', 'USDT', provider = binance, interval = '4-hour')

All caught up..!


In [5]:
btc.plot_candles()

In [6]:
# btc.start_stream()

In [7]:
# btc.stop_stream()

In [8]:
btc.plot_returns()

In [9]:
btc.plot_returns('hs')

In [10]:
btc.annualized_perf()

Return: 264.936% | Risk: 255.673%


In [11]:
from moonlander.strategy import TradingStrategyBase
from moonlander.indicators import SimpleMovingAverage

class SMACrossover(TradingStrategyBase):
    def setup(self):
        self.sma_short = SimpleMovingAverage(3)
        self.sma_long = SimpleMovingAverage(20, 'purple')
    
        self.indicators = [self.sma_short, self.sma_long]
    
    def logic(self, account, lookback):
        try:
            today = lookback.iloc[-1]
            
            sma_short_today = self.sma_short.lookback[-1]
            sma_long_today = self.sma_long.lookback[-1]
            sma_short_yesterday = self.sma_short.lookback[-2]
            sma_long_yesterday = self.sma_long.lookback[-2]
            
            # Buying
            buy_signal = (sma_short_today > sma_long_today) and (sma_short_yesterday < sma_long_yesterday)
            sell_signal = (sma_short_today < sma_long_today) and (sma_short_yesterday > sma_long_yesterday)
            if buy_signal:
                entry_price   = today.open
                entry_capital = account.buying_power
                account.buy(entry_capital=entry_capital, entry_price=entry_price, commission = 0.01)
                    
            # Selling
            elif sell_signal:
                exit_price = today.open
                account.sell(percent=1.0, current_price=exit_price, commission = 0.01)
                        
        except Exception as e:
            pass # Handles lookback errors in beginning of dataset

In [12]:
btc.data = btc.data.loc['2021-01-01 00:00:00':]

In [13]:
strat = SMACrossover(btc)

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
strat.backtest()


-------------- Results ----------------

Relative performance: -22.26%

Strategy     : 6626.63%
Net Profit   : 66266.29

Buy and Hold : 8523.6%
Net Profit   : 85235.98

Buys        : 30
Sells        : 29
--------------------
Total Trades : 59

---------------------------------------
